Diagnosing Student Code with Avicenna
===

In this Jupyter Notebook, we are exploring an interesting problem in educational technology and computer science education: the automated evaluation and diagnosis of student-submitted code. While automated systems can easily evaluate the correctness of code, diagnosing the _reasons_ behind any incorrect behavior can be a complex task.

### The Problem: Greatest Common Divisor (GCD)

Students have been assigned the task of writing a Python function to calculate the Greatest Common Divisor (GCD) of two integers. The GCD of two integers is the largest integer that divides both numbers without a remainder.

### Avicenna: Our Debugging and Diagnosis Tool

To aid us in this endeavor, we will be using Avicenna, an automated debugging tool that uses fuzz testing and symbolic reasoning to identify the conditions under which code behaves incorrectly. Avicenna's goal is to automatically diagnose software faults with high precision and recall, thereby making the debugging process more efficient.

With this notebook, we aim to:

1. Showcase how Avicenna can be used to diagnose student code.
2. Illustrate the capabilities of Avicenna through an example of diagnosing a GCD implementation.
3. Demonstrate how the diagnosis output can be used for generating new failure-inducing inputs.

By the end of this notebook, you'll have a clearer understanding of how automated diagnosis can aid in educational settings, and how tools like Avicenna contribute to this.

### Student Implementation

Below is the student's GCD implementation in Python.

In [1]:
# Student's implementation for calculating GCD
class Solution:
    def gcd(self, A, B):
        l=[]
        if A >= B:
            for i in range(1, A):
                if A % i == 0 and B % i == 0:
                    l.append(i)
            return l[-1] if len(l) > 1 else l[0]
        else:
            for i in range(1, B):
                if A % i == 0 and B % i == 0:
                    l.append(i)
            return l[-1] if len(l) > 1 else l[0]

### Reference Implementation

This is a reference implementation for calculating the GCD using the Euclidean algorithm.

In [2]:
# Reference implementation using the Euclidean algorithm
class Reference():
    def gcd(self, A, B):
        if A < B:
            A, B = B, A
        while B:
            A, B = B, A % B
        return A

### Testing the Implementations

Let's run some initial tests to compare the student and reference implementations.

In [3]:
# Comparing the two implementations
ob = Solution()
print(ob.gcd(10, 10))  # Output should be 10 for correct implementation

ref = Reference()
print(ref.gcd(10, 10))  # Output should be 10 as well

5
10


### Fuzzing Setup

Here we set up a grammar for fuzz testing using GrammarFuzzer.

In [4]:
# Setting up a grammar for fuzzing
from fuzzingbook.Grammars import Grammar
import string 

grammar: Grammar = {
    '<start>': ["<input>"],
    "<input>": ["<first> <second>"],
    "<first>": ["<integer>"],
    "<second>": ["<integer>"],
    "<integer>": ["<onenine><maybe_digits>"],
    "<onenine>": [str(num) for num in range(1, 10)],
    "<digit>": list(string.digits),
    "<maybe_digits>": ["", "<digits>"],
    "<digits>": ["<digit>", "<digit><digits>"],
}

### Oracle Construction

We will now use the reference implementation to construct an oracle for fuzz testing.

In [5]:
# Constructing an oracle using the reference GCD implementation
from avicenna.oracle_construction import construct_oracle, UnexpectedResultError, OracleResult

error_def = {UnexpectedResultError: OracleResult.BUG}
oracle = construct_oracle(ref.gcd, ob.gcd, error_def)

### Running the Fuzzer

Let's run the fuzzer and examine the results.

In [6]:
# Running the fuzzer to test the student's implementation
from fuzzingbook.GrammarFuzzer import GrammarFuzzer

fuzzer = GrammarFuzzer(grammar)
for _ in range(10):
    inp = fuzzer.fuzz()
    print(inp.ljust(30), oracle(inp))

455 873                        NO_BUG
7564 69922                     NO_BUG
3 56                           NO_BUG
5 4                            NO_BUG
34 764                         NO_BUG
962439 5                       NO_BUG
8 7                            NO_BUG
7 9                            NO_BUG
95 68                          NO_BUG
6 71                           NO_BUG


### Diagnosing Failures with Avicenna

We will now use Avicenna to automatically diagnose the conditions under which the student's implementation fails.

In [8]:
# Using Avicenna for diagnosis
from avicenna import Avicenna

initial_inputs = ["10 2", "4 4", "49 6618904968"]
avicenna = Avicenna(
    grammar,
    initial_inputs=initial_inputs,
    oracle=oracle
)

diagnosis = avicenna.explain()

### Viewing the Diagnosis

Let's interpret the diagnosis provided by Avicenna.

In [9]:
# Printing and interpreting the diagnosis
from isla.language import ISLaUnparser

print(f"Avicenna determined the following constraints to describe the failure circumstances:\n")
print(ISLaUnparser(diagnosis[0]).unparse())
print(f"Avicenna calculated a precision of {diagnosis[1]*100:.2f}% and a recall of {diagnosis[2]*100:.2f}%", end="\n\n")

Avicenna determined the following constraints to describe the failure circumstances:

(forall <input> container in start:
   exists <first> elem in container:
     (= (str.len elem) (str.to.int "1")) and
forall <input> container_0 in start:
  exists <second> elem_0 in container_0:
    (= (str.len elem_0) (str.to.int "1")))
Avicenna calculated a precision of 85.02% and a recall of 100.00%



## Use Self-Defined Patterns

In [10]:
error_def = {UnexpectedResultError: OracleResult.BUG}
oracle = construct_oracle(ref.gcd, ob.gcd, error_def)

In [11]:
patterns = [
"""forall <?NONTERMINAL> elem_1 in start:
  exists <?NONTERMINAL> elem_2 in start:
    (= (str.to.int elem_1) (str.to.int elem_2))""",
    """
    exists <?NONTERMINAL> elem in start:
    (>= (str.to.int elem) (str.to.int <?STRING>))
    """
]

In [12]:
# Using Avicenna for diagnosis
from avicenna import Avicenna

avicenna = Avicenna(
    grammar,
    initial_inputs=initial_inputs,
    oracle=oracle,
    patterns=patterns,
    #max_iterations=50,
)

diagnosis = avicenna.explain()

In [13]:
print(f"Avicenna determined the following constraints to describe the failure circumstances:\n")
print(ISLaUnparser(diagnosis[0]).unparse())
print(f"Avicenna calculated a precision of {diagnosis[1]*100:.2f}% and a recall of {diagnosis[2]*100:.2f}%", end="\n\n")

Avicenna determined the following constraints to describe the failure circumstances:

forall <integer> elem_1 in start:
  exists <first> elem_2 in start:
    (= (str.to.int elem_1) (str.to.int elem_2))
Avicenna calculated a precision of 100.00% and a recall of 100.00%

